In [ ]:
%%capture
!pip install yfinance --upgrade

In [ ]:
# import required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# required data
start_date = '2020-01-01'
end_date = '2024-12-31'

In [ ]:
# downloading assets
pepsi = yf.download("PEP", start=start_date, end=end_date)['Close']
coke = yf.download("KO", start=start_date, end=end_date)['Close']

In [ ]:
# creating a single dataframe by combining these two assets
df = pd.DataFrame()

df['pepsi'] = pepsi
df['coke'] = coke

df.reset_index(inplace=True)
df.head()

In [ ]:
# calculating spread and ratio

df['ratio'] = df['pepsi'] / df['coke']
df['spread'] = df['pepsi'] - df['coke']

In [ ]:
# calculating zscore by using mean and standard deviation from spread
def z_score(ser):
  return (ser - ser.mean()) /  ser.std()

df['z_score'] = z_score(df['spread'])

In [ ]:
df.head()

In [ ]:
# checking zscore distribution to determine entry_pos and exit pos  by using skewness
plt.hist(df["z_score"], bins=50)
plt.title("Z-Score Distribution")
plt.show()

In [ ]:
#entry_pos reduced to 1.5 from 2 due to too much skewed on positive 2 side
entry_pos = 1.5
exit_pos = -2

In [ ]:
# creating long and short positions by using zscore
df['long'] = df['z_score'] < -entry_pos
df['short'] = df['z_score'] > entry_pos

df['exit'] = abs(df['z_score']) < exit_pos

In [ ]:
df.head()

In [ ]:
# plotting z_score with thresholds

plt.figure(figsize=(10, 5))

plt.plot(df.Date, df["z_score"], label="z_score", color="blue") # z-score plot

plt.axhline(entry_pos, color="red", linestyle="--", label="short Threshold") # short_threshold
plt.axhline(-entry_pos, color="green", linestyle="--", label="long Threshold") #  long_threshold
plt.axhline(exit_pos, color="black", linestyle="dotted", label="Exit Threshold") #  exit_threshold
plt.axhline(-exit_pos, color="black", linestyle="dotted") # exit_threshold

plt.legend()

plt.title("z_score")

plt.show()

In [ ]:
# plotting of trading positions with prices of assets

plt.figure(figsize=(10, 5))

# pepsi and coke prices
plt.plot(df.Date, df["pepsi"], label="Pepsi", color="blue")
plt.plot(df.Date, df["coke"], label="Coke", color="orange")

# long entries of pepsi and cokae
plt.scatter(df.Date[df["long"]], df["pepsi"][df["long"]], color="green", label="Pepsi Long Entry", marker="^", s=100)
plt.scatter(df.Date[df["long"]], df["coke"][df["long"]], color="green", marker="^", s=100, edgecolors="black", label="Coke Long Entry")
plt.legend()

# short entries of pepsi and coke
plt.scatter(df.Date[df["short"]], df["pepsi"][df["short"]], color="red", label="Pepsi Short Entry", marker="v", s=100)
plt.scatter(df.Date[df["short"]], df["coke"][df["short"]], color="red", marker="v", s=100, edgecolors="black", label="Coke Short Entry")
plt.legend()

# Exits
# plt.scatter(df.Date[df["exit"]], df["pepsi"][df["exit"]], color="yellow", marker="o", s=150, edgecolors="black", label="Pepsi Exit")
# plt.scatter(df.Date[df["exit"]], df["coke"][df["exit"]], color="yellow", marker="o", s=150, edgecolors="red", label="Coke Exit")
# plt.legend()

plt.legend()

plt.title("Stock Prices with Trading Positions and Spread")

plt.xlabel("Date")
plt.ylabel("Stock Price")

plt.show()